論文  
https://arxiv.org/abs/2112.05142<br>
GitHub<br>
https://github.com/wty-ustc/HairCLIP<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/HairCLIP_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/wty-ustc/HairCLIP.git

!git clone https://github.com/omertov/encoder4editing.git
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

## ライブラリのインストール

In [ ]:
%cd /content/HairCLIP

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install tensorflow-io
!pip install --upgrade --no-cache-dir gdown

## ライブラリのインポート

In [ ]:
%cd /content/encoder4editing

from utils.alignment import align_face
from models.psp import pSp

In [ ]:
%cd /content/HairCLIP

import os
import gdown
from argparse import ArgumentParser

import sys
sys.path.append(".")
sys.path.append("..")
import tempfile
from argparse import Namespace

import dlib
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

import imageio
from IPython.display import HTML
from base64 import b64encode
import glob
import numpy as np
from PIL import Image
import random

from criteria.parse_related_loss import average_lab_color_loss

In [ ]:
%cd /content/HairCLIP/mapper

from mapper.datasets.latents_dataset_inference import LatentsDatasetInference
from mapper.hairclip_mapper import HairCLIPMapper

# 学習済みモデルのダウンロード
Access denied with the following error:<br>
が発生する場合、何回か実行

In [ ]:
%cd /content/HairCLIP

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

download_with_pydrive = True

class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive
        current_directory = os.getcwd()
        self.save_dir = "/content/HairCLIP/pretrained_models"
        os.makedirs(self.save_dir, exist_ok=True)
        if self.use_pydrive:
            self.authenticate()

    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)

    def download_file(self, file_id, file_name):
        file_dst = f'{self.save_dir}/{file_name}'
        if os.path.exists(file_dst):
            print(f'{file_name} already exists!')
            return
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id $file_id -O $file_dst

downloader = Downloader(download_with_pydrive)
downloader.download_file(file_id="1cUv_reLE6k3604or78EranS7XzuVMWeO", file_name="e4e_ffhq_encode.pt")

In [ ]:
%cd /content/HairCLIP

if not os.path.exists("./pretrained_models/hairclip.pt"):
  gdown.download('https://drive.google.com/uc?id=1hqZT6ZMldhX3M_x378Sm4Z2HMYr-UwQ4', "./pretrained_models/hairclip.pt", quiet=False)
if not os.path.exists("./pretrained_models/stylegan2-ffhq-config-f.pt"):
  gdown.download('https://drive.google.com/uc?id=1pts5tkfAcWrg4TpLDu6ILF5wHID32Nzm', "./pretrained_models/stylegan2-ffhq-config-f.pt", quiet=False)
if not os.path.exists("./pretrained_models/model_ir_se50.pth"):
  gdown.download('https://drive.google.com/uc?id=1FS2V756j-4kWduGxfir55cMni5mZvBTv', "./pretrained_models/model_ir_se50.pth", quiet=False)


# if not os.path.exists("./pretrained_models/test_faces.pt"):
#   gdown.download('https://drive.google.com/uc?id=1j7RIfmrCoisxx3t-r-KC02Qc8barBecr', "./pretrained_models/test_faces.pt", quiet=False)

if not os.path.exists("./pretrained_models/shape_predictor_68_face_landmarks.dat.bz2"):
  !wget -c http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 \
        -O ./pretrained_models/shape_predictor_68_face_landmarks.dat.bz2
  !bzip2 -dk ./pretrained_models/shape_predictor_68_face_landmarks.dat.bz2

# テスト画像のセットアップ

## テスト画像のダウンロード
[使用画像](https://www.pakutaso.com/20190117010post-18492.html)

In [ ]:
%cd /content/HairCLIP
!mkdir demo

!wget -c https://www.pakutaso.com/shared/img/thumb/model10211041_TP_V4.jpg \
      -O ./demo/model10211041_TP_V4.jpg

# Edit Hair

## Set parametor

In [ ]:
%cd /content/HairCLIP

# @markdown 入力画像
image_path = "/content/HairCLIP/demo/model10211041_TP_V4.jpg" #@param {type:"string"}


# @markdown editing_type="both"でrandomに生成<br>
# @markdown randomの場合下記の設定は反映されません。
IsRandom = True #@param {type:"boolean"}
random_num = 30 #@param {type:"integer"}

# @markdown randomではない場合は以下設定<br>
# @markdown 編集タイプ colorのみ、styleのみ、両方
editing_type = "both" #@param["hairstyle", "color", "both"]
# @markdown 髪型選択
hairstyle_description = "crew cut hairstyle" #@param["afro hairstyle", "bob cut hairstyle", "bowl cut hairstyle", "braid hairstyle", "caesar cut hairstyle", "chignon hairstyle", "cornrows hairstyle", "crew cut hairstyle", "crown braid hairstyle", "curtained hair hairstyle", "dido flip hairstyle", "dreadlocks hairstyle", "extensions hairstyle", "fade hairstyle", "fauxhawk hairstyle", "finger waves hairstyle", "french braid hairstyle", "frosted tips hairstyle", "full crown hairstyle", "harvard clip hairstyle", "high and tight hairstyle", "hime cut hairstyle", "hi-top fade hairstyle","jewfro hairstyle", "jheri curl hairstyle", "liberty spikes hairstyle", "marcel waves hairstyle", "mohawk hairstyle", "pageboy hairstyle", "perm hairstyle", "pixie cut hairstyle", "psychobilly wedge hairstyle", "quiff hairstyle", "regular taper cut hairstyle", "ringlets hairstyle", "shingle bob hairstyle", "short hair hairstyle", "slicked-back hairstyle", "spiky hair hairstyle","surfer hair hairstyle", "taper cut hairstyle", "the rachel hairstyle", "undercut hairstyle", "updo hairstyle"]
# @markdown 髪色選択
color_description = "yellow" #@param["purple", "red", "orange", "yellow", "green", "blue", "gray", "brown", "black", "white", "blond", "pink"]

# 出力先ディレクトリ作成
!mkdir outputs

## Define functions

In [ ]:
def run_alignment(image_path):
  predictor = dlib.shape_predictor("/content/HairCLIP/pretrained_models/shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor)
  print("Aligned image has shape: {}".format(aligned_image.size))
  return aligned_image

In [ ]:
def run_on_batch_e4e(inputs, net):
  images, latents = net(
      inputs.to("cuda").float(), randomize_noise=False, return_latents=True
      )
  return images, latents

In [ ]:
def run_on_batch(
    inputs,
    hairstyle_text_inputs,
    color_text_inputs,
    hairstyle_tensor_hairmasked,
    color_tensor_hairmasked,
    net,
):
    w = inputs
    with torch.no_grad():
        w_hat = w + 0.1 * net.mapper(
            w,
            hairstyle_text_inputs,
            color_text_inputs,
            hairstyle_tensor_hairmasked,
            color_tensor_hairmasked,
        )
        x_hat, w_hat = net.decoder(
            [w_hat],
            input_is_latent=True,
            return_latents=True,
            randomize_noise=False,
            truncation=1,
        )
        x, _ = net.decoder(
            [w], input_is_latent=True, randomize_noise=False, truncation=1
        )
        result_batch = (x_hat, w_hat, x)
    return result_batch


In [ ]:
def predict(
    edit_t, hair_d, color_d, 
    ck, im_path, trans, device):
    editing_type_ = edit_t
    hairstyle_description_ = hair_d
    color_description_ = color_d

    if editing_type_ == "both":
      assert (
          hairstyle_description_ is not None and color_d is not None
          ), ("Please provide description " "for both hairstyle and color.")
    elif editing_type_ == "hairstyle":
      assert (
          hairstyle_description_ is not None
          ), "Please provide description for hairstyle."
    else:
      assert (
          color_description_ is not None
          ), "Please provide description for color."

    opts = ck["opts"]
    opts = Namespace(**opts)
    opts.editing_type = editing_type_
    opts.input_type = "text"
    opts.color_description = color_description_

    if hair_d is not None:
      with open("/content/HairCLIP/outputs/hairstyle_description.txt", "w") as file:
        file.write(hairstyle_description_)
      opts.hairstyle_description = "/content/HairCLIP/outputs/hairstyle_description.txt"

    opts.checkpoint_path  = "/content/HairCLIP/pretrained_models/hairclip.pt"
    opts.parsenet_weights = "/content/HairCLIP/pretrained_models/parsenet.pth"
    opts.stylegan_weights = "/content/HairCLIP/pretrained_models/stylegan2-ffhq-config-f.pt"
    opts.ir_se50_weights  = "/content/HairCLIP/pretrained_models/model_ir_se50.pth"
    net = HairCLIPMapper(opts)
    net.eval()
    net.cuda()

    # 顔部分のalignment, transform
    input_image = run_alignment(str(im_path))
    resize_dims = (256, 256)
    input_image.resize(resize_dims)
    transformed_image = trans(input_image)

    with torch.no_grad():
      images, latents = run_on_batch_e4e(
          transformed_image.unsqueeze(0), e4e_net
          )
      print("Latent code calculated!")

    dataset = LatentsDatasetInference(latents=latents.cpu(), opts=opts)
    dataloader = DataLoader(dataset)

    average_color_loss = (
        average_lab_color_loss.AvgLabLoss(opts).to(device).eval()
        )

    out_filename = editing_type_ + "_" \
      + hairstyle_description_ + "_" \
      + color_description_ + ".png"
    out_path = os.path.join("/content/HairCLIP/outputs", out_filename)
    print("output path:", out_path)

    for input_batch in tqdm(dataloader):
      with torch.no_grad():
        w,hairstyle_text_inputs_list, color_text_inputs_list, selected_description_tuple_list, hairstyle_tensor_list, color_tensor_list = input_batch

        hairstyle_text_inputs = hairstyle_text_inputs_list[0]
        color_text_inputs = color_text_inputs_list[0]

        selected_description = selected_description_tuple_list[0][0]
        hairstyle_tensor = hairstyle_tensor_list[0]
        color_tensor = color_tensor_list[0]

        w = w.cuda().float()
        hairstyle_text_inputs = hairstyle_text_inputs.cuda()
        color_text_inputs = color_text_inputs.cuda()
        hairstyle_tensor = hairstyle_tensor.cuda()
        color_tensor = color_tensor.cuda()
        if hairstyle_tensor.shape[1] != 1:
          hairstyle_tensor_hairmasked = (
              hairstyle_tensor * average_color_loss.gen_hair_mask(hairstyle_tensor))
        else:
          hairstyle_tensor_hairmasked = torch.Tensor([0]).unsqueeze(0).cuda()

        if color_tensor.shape[1] != 1:
          color_tensor_hairmasked = (
              color_tensor * average_color_loss.gen_hair_mask(color_tensor))
        else:
          color_tensor_hairmasked = torch.Tensor([0]).unsqueeze(0).cuda()
          result_batch = run_on_batch(
              w, hairstyle_text_inputs, color_text_inputs,
              hairstyle_tensor_hairmasked, color_tensor_hairmasked, net,)   

        if (hairstyle_tensor.shape[1] != 1) and (color_tensor.shape[1] != 1):
          img_tensor = torch.cat([hairstyle_tensor, color_tensor], dim=3)
        elif hairstyle_tensor.shape[1] != 1:
          img_tensor = hairstyle_tensor
        elif color_tensor.shape[1] != 1:
          img_tensor = color_tensor
        else:
          img_tensor = None   

        if img_tensor is not None:
          if img_tensor.shape[3] == 1024:
            couple_output = torch.cat(
                [result_batch[2][0].unsqueeze(0), result_batch[0][0].unsqueeze(0), img_tensor,])
          elif img_tensor.shape[3] == 2048:
            couple_output = torch.cat(
                [result_batch[2][0].unsqueeze(0), result_batch[0][0].unsqueeze(0),
                 img_tensor[:, :, :, 0:1024], img_tensor[:, :, :, 1024::], ])
            couple_output = torch.cat(
                [result_batch[2][0].unsqueeze(0), result_batch[0][0].unsqueeze(0),
                 img_tensor[:, :, :, 0:1024], img_tensor[:, :, :, 1024::], ])
        else:
            couple_output = torch.cat(
                [result_batch[2][0].unsqueeze(0),result_batch[0][0].unsqueeze(0),])

        torchvision.utils.save_image(
            couple_output, str(out_path), normalize=True, range=(-1, 1))

## Predict

In [ ]:
with open("/content/HairCLIP/mapper/hairstyle_list.txt") as infile:
  HAIRSTYLE_LIST = sorted([line.rstrip() for line in infile])
COLORSTYLE_LIST = ["purple", "red", "orange", "yellow", "green", "blue", "gray", "brown", "black", "white", "blond", "pink"]

device = "cuda:0"

# load e4e ffhq model
e4e_model_path = "/content/HairCLIP/pretrained_models/e4e_ffhq_encode.pt"
e4e_ckpt = torch.load(e4e_model_path, map_location="cpu")
e4e_opts = e4e_ckpt["opts"]
e4e_opts["checkpoint_path"] = e4e_model_path
e4e_opts = Namespace(**e4e_opts)

e4e_net = pSp(e4e_opts)
e4e_net.eval()
e4e_net.cuda()
print("e4e model successfully loaded!")

# set transforms
img_transforms = transforms.Compose(
    [
     transforms.Resize((256, 256)),
     transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

# load hairclip model
checkpoint_path = "/content/HairCLIP/pretrained_models/hairclip.pt"
ckpt = torch.load(checkpoint_path, map_location="cpu")

if IsRandom == True:
  for i in range(random_num):
    hairstyle_index = random.randrange(0, (len(HAIRSTYLE_LIST)-1), 1)
    colorstyle_index = random.randrange(0, (len(COLORSTYLE_LIST)-1), 1)
    predict(
        "both", HAIRSTYLE_LIST[hairstyle_index], COLORSTYLE_LIST[colorstyle_index], 
        ckpt, image_path, img_transforms, device)

else:
  predict(
      editing_type, hairstyle_description, color_description, 
      ckpt, image_path, img_transforms, device)

## Show result

In [ ]:
def generate_mp4(out_name, images, kwargs):
  writer = imageio.get_writer(out_name + '.mp4', **kwargs)
  for image in images:
    writer.append_data(image)
  writer.close()

def show_mp4(filename, width):
  mp4 = open(filename + '.mp4', 'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  display(HTML("""
  <video width="%d" controls autoplay loop>
    <source src="%s" type="video/mp4">
  </video>
  """ % (width, data_url)))

res_list = glob.glob("/content/HairCLIP/outputs/*.png")
images = []
for img_path in res_list:
  images.append(np.array(Image.open(img_path)))

kwargs = {'fps': 2}

gif_path = os.path.join("/content/HairCLIP/outputs", "animation")
generate_mp4(gif_path, images, kwargs)
show_mp4(gif_path, width=514)